# Import Non-Imputed data for Categorical Modeling

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
cd ../Desktop/Insight/2012_MEPS/

/Users/bstankev/Desktop/Insight/2012_MEPS


In [17]:
df = pd.read_csv('feat_new.csv')

In [18]:
df.head()

,Unnamed: 0,Unnamed: 0.1,DUID_x,PID_x,DUPERSID,OBVTCH12,OBVEXP12,ERTTCH12,ERTEXP12,IPTTCH12,...,ADHDADDX,ANYLIM12,ADSMOK42,ADMALS42,ADMWLM42,ADPAIN42,ADCAPE42,ADNRGY42,ADDOWN42,ADSOCA42
0,0,0,20004,101,20004101,NaN,NaN,NaN,NaN,NaN,...,0,2,2,3,3,1,2,3,4,5
1,1,1,20004,102,20004102,NaN,NaN,NaN,NaN,NaN,...,0,2,2,3,5,1,3,3,4,5
2,2,2,20004,103,20004103,NaN,NaN,NaN,NaN,NaN,...,2,2,0,0,0,0,0,0,0,0
3,3,3,20005,101,20005101,1100,451,NaN,NaN,NaN,...,0,1,2,1,5,5,3,4,2,2
4,4,4,20005,102,20005102,490,127,NaN,NaN,NaN,...,0,2,2,5,5,1,1,1,5,5


In [19]:
len(df)

38974

# Create Categorical columns for spend or not (yes, no)

In [20]:
df['OBVEXPcat'] = np.where(df['OBVEXP12']>0, 1, 0)
df['ERTEXPcat'] = np.where(df['ERTEXP12']>0, 1, 0)
df['HHAEXPcat'] = np.where(df['HHAEXP12']>0, 1, 0)
df['IPTEXPcat'] = np.where(df['IPTEXP12']>0, 1, 0)
df['RXEXPcat'] = np.where(df['RXEXP12']>0, 1, 0)
df['TOTEXPcat'] = np.where(df['TOTEXP12']>0, 1, 0)

In [21]:
df.head()

,Unnamed: 0,Unnamed: 0.1,DUID_x,PID_x,DUPERSID,OBVTCH12,OBVEXP12,ERTTCH12,ERTEXP12,IPTTCH12,...,ADCAPE42,ADNRGY42,ADDOWN42,ADSOCA42,OBVEXPcat,ERTEXPcat,HHAEXPcat,IPTEXPcat,RXEXPcat,TOTEXPcat
0,0,0,20004,101,20004101,NaN,NaN,NaN,NaN,NaN,...,2,3,4,5,0,0,0,0,0,0
1,1,1,20004,102,20004102,NaN,NaN,NaN,NaN,NaN,...,3,3,4,5,0,0,0,0,0,0
2,2,2,20004,103,20004103,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1
3,3,3,20005,101,20005101,1100,451,NaN,NaN,NaN,...,3,4,2,2,1,0,0,0,1,1
4,4,4,20005,102,20005102,490,127,NaN,NaN,NaN,...,1,1,5,5,1,0,0,0,0,1


# Create Feature & class arrays for each expenditure type

Taking into account best features for each model

### Office based visits

In [22]:
obX = df[['POVLEV12','HLTHSTAT',"BMINDX53","HIBPAGED",
            "STRKAGED",'MNHLTHSTAT',"DIABAGED",'AGELAST',
            "ADPAIN42","CHOLAGED", "ADCAPE42", 'EDRECODE',
            "ADNRGY42"]]
obX0 = obX.fillna(0)

obY = df['OBVEXPcat']

### Scale continuous & categorical (ordinal) features

### Office-based visits

In [23]:
# re-scale (normalize) continuous  & categorical (ordinal) data
from sklearn import preprocessing

obX1 = preprocessing.StandardScaler().fit(obX0)
obX1 = obX1.transform(obX0)

In [24]:
obY.unique()

array([0, 1])

### Model Round 1 - LogisticRegression

### Office-based visits

In [25]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

rng = np.random.RandomState(42)
obx_train, obx_test, oby_train, oby_test = train_test_split(obX1,obY, random_state=rng)

model = LogisticRegression()
model.fit(obx_train, oby_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', random_state=None, tol=0.0001)

In [26]:
# make predictions
expected = oby_test
predicted = model.predict(obx_test)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

             precision    recall  f1-score   support

          0       0.60      0.38      0.46      3612
          1       0.70      0.85      0.77      6132

avg / total       0.66      0.68      0.65      9744

[[1356 2256]
 [ 899 5233]]


In [29]:
print model.predict_proba(obx_test)

[[ 0.24102949  0.75897051]
 [ 0.5114778   0.4885222 ]
 [ 0.62098244  0.37901756]
 ..., 
 [ 0.37211228  0.62788772]
 [ 0.18691837  0.81308163]
 [ 0.3726148   0.6273852 ]]


In [32]:
score = metrics.accuracy_score(oby_test, predicted)
print("accuracy:   %0.3f" % score)

accuracy:   0.676


In [143]:
df.groupby('OBVEXPcat').count()

,Unnamed: 0,Unnamed: 0.1,DUID_x,PID_x,DUPERSID,OBVTCH12,OBVEXP12,ERTTCH12,ERTEXP12,IPTTCH12,...,ADPAIN42,ADCAPE42,ADNRGY42,ADDOWN42,ADSOCA42,ERTEXPcat,HHAEXPcat,IPTEXPcat,RXEXPcat,TOTEXPcat
OBVEXPcat,,,,,,,,,,,,,,,,,,,,,
0,14700,14700,14700,14700,14700,322,0,1007,926,259,...,14700,14700,14700,14700,14700,14700,14700,14700,14700,14700
1,24274,24274,24274,24274,24274,24274,24274,3792,3701,2121,...,24274,24274,24274,24274,24274,24274,24274,24274,24274,24274


In [144]:
from sklearn.ensemble import RandomForestClassifier

# split the data
xro_train, xro_test, yro_train, yro_test = train_test_split(obX1, obY, random_state=42)

sw = np.array([2 if i == 0 else 1 for i in yro_train])

ro2 = RandomForestClassifier(n_estimators=100, oob_score=True,n_jobs=100, 
                              max_features=0.5, max_depth=7, min_samples_leaf=2)
ro2.fit(xro_train,yro_train,sample_weight=sw)

RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=7, max_features=0.5,
            max_leaf_nodes=None, min_density=None, min_samples_leaf=2,
            min_samples_split=2, n_estimators=100, n_jobs=100,
            oob_score=True, random_state=None, verbose=0)

In [145]:
# make predictions
expected = yro_test
predicted = ro2.predict(xro_test)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))
score = metrics.accuracy_score(yro_test, predicted)
print("accuracy:   %0.3f" % score)

             precision    recall  f1-score   support

          0       0.50      0.81      0.62      3612
          1       0.83      0.53      0.64      6132

avg / total       0.71      0.63      0.64      9744

[[2930  682]
 [2898 3234]]
accuracy:   0.633


### ER Visits

In [35]:
erX = df[['POVLEV12','HLTHSTAT',"BMINDX53","HIBPAGED",
            "STRKAGED",'MNHLTHSTAT',"DIABAGED",'AGELAST',
            "ADPAIN42","CHOLAGED", "ADCAPE42", 'EDRECODE',
            "ADNRGY42",'ACTDTY53','MARRY12X']]
erX0 = erX.fillna(0)
# re-scale (normalize) continuous  & categorical (ordinal) data
erX1 = preprocessing.StandardScaler().fit(erX0[['POVLEV12','HLTHSTAT',"BMINDX53","HIBPAGED",
            "STRKAGED",'MNHLTHSTAT',"DIABAGED",'AGELAST',
            "ADPAIN42","CHOLAGED", "ADCAPE42", 'EDRECODE',
            "ADNRGY42"]])
erX1 = erX1.transform(erX0[['POVLEV12','HLTHSTAT',"BMINDX53","HIBPAGED",
            "STRKAGED",'MNHLTHSTAT',"DIABAGED",'AGELAST',
            "ADPAIN42","CHOLAGED", "ADCAPE42", 'EDRECODE',
            "ADNRGY42"]])

erX2 = np.append(erX1,erX0[['ACTDTY53','MARRY12X']], axis=1)


erY = df['ERTEXPcat']

In [36]:
rng = np.random.RandomState(42)
erx_train, erx_test, ery_train, ery_test = train_test_split(erX2,erY, random_state=rng)

model = LogisticRegression()
model.fit(erx_train, ery_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', random_state=None, tol=0.0001)

In [37]:
# make predictions
expected = ery_test
predicted = model.predict(erx_test)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

             precision    recall  f1-score   support

          0       0.88      1.00      0.94      8604
          1       0.32      0.01      0.02      1140

avg / total       0.82      0.88      0.83      9744

[[8576   28]
 [1127   13]]


In [38]:
score = metrics.accuracy_score(ery_test, predicted)
print("accuracy:   %0.3f" % score)

accuracy:   0.881


In [137]:
df.groupby('ERTEXPcat').count()

,Unnamed: 0,Unnamed: 0.1,DUID_x,PID_x,DUPERSID,OBVTCH12,OBVEXP12,ERTTCH12,ERTEXP12,IPTTCH12,...,ADPAIN42,ADCAPE42,ADNRGY42,ADDOWN42,ADSOCA42,OBVEXPcat,HHAEXPcat,IPTEXPcat,RXEXPcat,TOTEXPcat
ERTEXPcat,,,,,,,,,,,,,,,,,,,,,
0,34347,34347,34347,34347,34347,20859,20573,172,0,1275,...,34347,34347,34347,34347,34347,34347,34347,34347,34347,34347
1,4627,4627,4627,4627,4627,3737,3701,4627,4627,1105,...,4627,4627,4627,4627,4627,4627,4627,4627,4627,4627


In [141]:
from sklearn.ensemble import RandomForestClassifier

# split the data
xre_train, xre_test, yre_train, yre_test = train_test_split(erX2, erY, random_state=42)

sw = np.array([7 if i == 1 else 1 for i in yre_train])

re2 = RandomForestClassifier(n_estimators=100, oob_score=True,n_jobs=100, 
                              max_features=0.5, max_depth=7, min_samples_leaf=2)
re2.fit(xre_train,yre_train,sample_weight=sw)

RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=7, max_features=0.5,
            max_leaf_nodes=None, min_density=None, min_samples_leaf=2,
            min_samples_split=2, n_estimators=100, n_jobs=100,
            oob_score=True, random_state=None, verbose=0)

In [142]:
# make predictions
expected = yre_test
predicted = re2.predict(xre_test)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))
score = metrics.accuracy_score(yre_test, predicted)
print("accuracy:   %0.3f" % score)

             precision    recall  f1-score   support

          0       0.92      0.78      0.84      8604
          1       0.22      0.45      0.29      1140

avg / total       0.83      0.74      0.78      9744

[[6722 1882]
 [ 622  518]]
accuracy:   0.743


### RX 

In [41]:
rxX = df[['POVLEV12','HLTHSTAT',"BMINDX53","HIBPAGED",
            "STRKAGED",'MNHLTHSTAT',"DIABAGED",'AGELAST',
            "CHOLAGED", 'EDRECODE',"ADNRGY42",'ANYLIM12', 
            'ADHDAGED','HIDEG','ADPAIN42','RACEV1X']]
rxX0 = rxX.fillna(0)

columns = ['POVLEV12','HLTHSTAT',"BMINDX53","HIBPAGED",
            "STRKAGED",'MNHLTHSTAT',"DIABAGED",'AGELAST',
            "CHOLAGED", 'EDRECODE',"ADNRGY42", 'ADHDAGED',
            'ADPAIN42']
#'ANYLIM12',,'HIDEG','RACEV1X'
# re-scale (normalize) continuous  & categorical (ordinal) data
rxX1 = preprocessing.StandardScaler().fit(rxX0[columns])
rxX1 = rxX1.transform(rxX0[columns])

rxX2 = np.append(rxX1,rxX0[['ANYLIM12','HIDEG','RACEV1X']], axis=1)

rxY = df['RXEXPcat']

In [42]:
rng = np.random.RandomState(42)
rxx_train, rxx_test, rxy_train, rxy_test = train_test_split(rxX2,rxY, random_state=rng)

model = LogisticRegression()
model.fit(rxx_train, rxy_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', random_state=None, tol=0.0001)

In [43]:
# make predictions
expected = rxy_test
predicted = model.predict(rxx_test)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

             precision    recall  f1-score   support

          0       0.66      0.79      0.72      4638
          1       0.77      0.62      0.69      5106

avg / total       0.71      0.70      0.70      9744

[[3674  964]
 [1929 3177]]


In [44]:
score = metrics.accuracy_score(rxy_test, predicted)
print("accuracy:   %0.3f" % score)

accuracy:   0.703


### Inpatient Visits

In [46]:
inX = df[['POVLEV12','HLTHSTAT',"BMINDX53","HIBPAGED",
            "STRKAGED",'MNHLTHSTAT',"DIABAGED",'AGELAST',
            "ADPAIN42","CHOLAGED", "ADCAPE42", 'EDRECODE',
            "ADNRGY42",'HIDEG','SEX','ANYLIM12', 'HISPCAT']]
inX0 = inX.fillna(0)

columns = ['POVLEV12','HLTHSTAT',"BMINDX53","HIBPAGED",
            "STRKAGED",'MNHLTHSTAT',"DIABAGED",'AGELAST',
            "ADPAIN42","CHOLAGED", "ADCAPE42", 'EDRECODE',
            "ADNRGY42"]

#'HIDEG','SEX','ANYLIM12', 'HISPCAT'
# re-scale (normalize) continuous  & categorical (ordinal) data
inX1 = preprocessing.StandardScaler().fit(inX0[columns])
inX1 = inX1.transform(inX0[columns])

inX2 = np.append(inX1,inX0[['HIDEG','SEX','ANYLIM12', 'HISPCAT']], axis=1)

inY = df['IPTEXPcat']




In [138]:
from sklearn.ensemble import RandomForestClassifier

# split the data
xri_train, xri_test, yri_train, yri_test = train_test_split(inX2, inY, random_state=42)

sw = np.array([15 if i == 1 else 1 for i in yri_train])

ri2 = RandomForestClassifier(n_estimators=100, oob_score=True,n_jobs=100, 
                              max_features=0.5, max_depth=7, min_samples_leaf=2)
ri2.fit(xri_train,yri_train,sample_weight=sw)

RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=7, max_features=0.5,
            max_leaf_nodes=None, min_density=None, min_samples_leaf=2,
            min_samples_split=2, n_estimators=100, n_jobs=100,
            oob_score=True, random_state=None, verbose=0)

In [139]:
# make predictions
expected = yri_test
predicted = ri2.predict(xri_test)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))


             precision    recall  f1-score   support

          0       0.97      0.82      0.89      9149
          1       0.17      0.58      0.27       595

avg / total       0.92      0.80      0.85      9744

[[7499 1650]
 [ 251  344]]


In [140]:
score = metrics.accuracy_score(yri_test, predicted)
print("accuracy:   %0.3f" % score)

accuracy:   0.805


In [130]:
df.groupby('IPTEXPcat').count()

,Unnamed: 0,Unnamed: 0.1,DUID_x,PID_x,DUPERSID,OBVTCH12,OBVEXP12,ERTTCH12,ERTEXP12,IPTTCH12,...,ADPAIN42,ADCAPE42,ADNRGY42,ADDOWN42,ADSOCA42,OBVEXPcat,ERTEXPcat,HHAEXPcat,RXEXPcat,TOTEXPcat
IPTEXPcat,,,,,,,,,,,,,,,,,,,,,
0,36613,36613,36613,36613,36613,22478,22169,3684,3532,19,...,36613,36613,36613,36613,36613,36613,36613,36613,36613,36613
1,2361,2361,2361,2361,2361,2118,2105,1115,1095,2361,...,2361,2361,2361,2361,2361,2361,2361,2361,2361,2361


In [47]:
rng = np.random.RandomState(42)
inx_train, inx_test, iny_train, iny_test = train_test_split(inX2,inY, random_state=rng)

model = LogisticRegression()
model.fit(inx_train, iny_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', random_state=None, tol=0.0001)

In [48]:
# make predictions
expected = iny_test
predicted = model.predict(inx_test)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

             precision    recall  f1-score   support

          0       0.94      1.00      0.97      9149
          1       0.53      0.02      0.03       595

avg / total       0.91      0.94      0.91      9744

[[9141    8]
 [ 586    9]]


In [50]:
score = metrics.accuracy_score(iny_test, predicted)
print("accuracy:   %0.3f" % score)

accuracy:   0.939


### Home Health

In [52]:
hhX = df[['POVLEV12','HLTHSTAT',"BMINDX53","HIBPAGED",
            "STRKAGED",'MNHLTHSTAT',"DIABAGED",'AGELAST',
            "CHOLAGED", "ADCAPE42", 'EDRECODE',"ADNRGY42",'MARRY12X', 'RACEV1X']]
hhX0 = hhX.fillna(0)

columns = ['POVLEV12','HLTHSTAT',"BMINDX53","HIBPAGED",
            "STRKAGED",'MNHLTHSTAT',"DIABAGED",'AGELAST',
            "CHOLAGED", "ADCAPE42", 'EDRECODE',"ADNRGY42",
            ]

#'HIDEG','SEX','ANYLIM12', 'HISPCAT'
# re-scale (normalize) continuous  & categorical (ordinal) data
hhX1 = preprocessing.StandardScaler().fit(hhX0[columns])
hhX1 = hhX1.transform(hhX0[columns])

hhX2 = np.append(hhX1,hhX0[['MARRY12X', 'RACEV1X']], axis=1)

hhY = df['HHAEXPcat']


In [100]:
df.groupby('HHAEXPcat').count()

,Unnamed: 0,Unnamed: 0.1,DUID_x,PID_x,DUPERSID,OBVTCH12,OBVEXP12,ERTTCH12,ERTEXP12,IPTTCH12,...,ADPAIN42,ADCAPE42,ADNRGY42,ADDOWN42,ADSOCA42,OBVEXPcat,ERTEXPcat,IPTEXPcat,RXEXPcat,TOTEXPcat
HHAEXPcat,,,,,,,,,,,,,,,,,,,,,
0,38307,38307,38307,38307,38307,23981,23660,4561,4389,2049,...,38307,38307,38307,38307,38307,38307,38307,38307,38307,38307
1,667,667,667,667,667,615,614,238,238,331,...,667,667,667,667,667,667,667,667,667,667


In [53]:
rng = np.random.RandomState(42)
hhx_train, hhx_test, hhy_train, hhy_test = train_test_split(hhX2,hhY, random_state=rng)

model = LogisticRegression()
model.fit(hhx_train, hhy_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', random_state=None, tol=0.0001)

In [106]:
from sklearn.ensemble import RandomForestClassifier

# split the data
xrf_train, xrf_test, yrf_train, yrf_test = train_test_split(hhX2, hhY, random_state=42)

sw = np.array([57 if i == 1 else 1 for i in yrf_train])

rf2 = RandomForestClassifier(n_estimators=100, oob_score=True,n_jobs=100, 
                              max_features=0.5, max_depth=7, min_samples_leaf=2)
rf2.fit(xrf_train,yrf_train,sample_weight=sw)

RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=7, max_features=0.5,
            max_leaf_nodes=None, min_density=None, min_samples_leaf=2,
            min_samples_split=2, n_estimators=100, n_jobs=100,
            oob_score=True, random_state=None, verbose=0)

In [119]:
xrf_test.shape

(9744, 14)

In [129]:
# make predictions
expected = yrf_test
predicted = rf2.predict(xrf_test)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))


             precision    recall  f1-score   support

          0       0.99      0.93      0.96      9590
          1       0.12      0.60      0.19       154

avg / total       0.98      0.92      0.95      9744

[[8884  706]
 [  62   92]]


In [128]:
score = metrics.accuracy_score(yrf_test, predicted)
print("accuracy:   %0.3f" % score)

accuracy:   0.921


In [54]:
# make predictions
expected = hhy_test
predicted = model.predict(hhx_test)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

             precision    recall  f1-score   support

          0       0.98      1.00      0.99      9590
          1       0.58      0.05      0.08       154

avg / total       0.98      0.98      0.98      9744

[[9585    5]
 [ 147    7]]


In [55]:
score = metrics.accuracy_score(iny_test, predicted)
print("accuracy:   %0.3f" % score)

accuracy:   0.939


# Total

In [86]:
totX = df[['MNHLTHSTAT','HLTHSTAT', 'AGELAST',
            'POVLEV12','EDRECODE',"HIBPAGED", 
            "STRKAGED","CHOLAGED", "DIABAGED", "JTPAIN31",
            "BMINDX53","ADMALS42", "ADMWLM42","ADPAIN42",  
            "ADNRGY42"]]
totX0 = totX.fillna(0)

# re-scale (normalize) continuous  & categorical (ordinal) data
totX1 = preprocessing.StandardScaler().fit(totX0)
totX1 = totX1.transform(totX0)

subX = df[['HIDEG', 'SEX']].fillna(0)
totX2 = np.append(totX1,subX, axis=1)


totY = df['TOTEXPcat'].values

In [88]:
rng = np.random.RandomState(42)
totx_train, totx_test, toty_train, toty_test = train_test_split(totX2,totY, random_state=rng)

model = LogisticRegression()
model.fit(totx_train, toty_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', random_state=None, tol=0.0001)

In [89]:
# make predictions
expected = toty_test
predicted = model.predict(totx_test)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

             precision    recall  f1-score   support

          0       0.60      0.20      0.30      2274
          1       0.80      0.96      0.87      7470

avg / total       0.75      0.78      0.74      9744

[[ 465 1809]
 [ 313 7157]]


In [90]:
score = metrics.accuracy_score(toty_test, predicted)
print("accuracy:   %0.3f" % score)

accuracy:   0.782


In [ ]:
#sample_weight = np.array([5 if i == 0 else 1 for i in y])

In [146]:
df.groupby('TOTEXPcat').count()

,Unnamed: 0,Unnamed: 0.1,DUID_x,PID_x,DUPERSID,OBVTCH12,OBVEXP12,ERTTCH12,ERTEXP12,IPTTCH12,...,ADPAIN42,ADCAPE42,ADNRGY42,ADDOWN42,ADSOCA42,OBVEXPcat,ERTEXPcat,HHAEXPcat,IPTEXPcat,RXEXPcat
TOTEXPcat,,,,,,,,,,,,,,,,,,,,,
0,9167,9167,9167,9167,9167,122,0,24,0,0,...,9167,9167,9167,9167,9167,9167,9167,9167,9167,9167
1,29807,29807,29807,29807,29807,24474,24274,4775,4627,2380,...,29807,29807,29807,29807,29807,29807,29807,29807,29807,29807


In [147]:
from sklearn.ensemble import RandomForestClassifier

# split the data
xrt_train, xrt_test, yrt_train, yrt_test = train_test_split(totX2, totY, random_state=42)

sw = np.array([3 if i == 0 else 1 for i in yrt_train])

rt2 = RandomForestClassifier(n_estimators=100, oob_score=True,n_jobs=100, 
                              max_features=0.5, max_depth=7, min_samples_leaf=2)
rt2.fit(xrt_train,yrt_train,sample_weight=sw)

RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=7, max_features=0.5,
            max_leaf_nodes=None, min_density=None, min_samples_leaf=2,
            min_samples_split=2, n_estimators=100, n_jobs=100,
            oob_score=True, random_state=None, verbose=0)

In [148]:
# make predictions
expected = yrt_test
predicted = rt2.predict(xrt_test)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))


             precision    recall  f1-score   support

          0       0.43      0.67      0.52      2274
          1       0.88      0.73      0.80      7470

avg / total       0.77      0.71      0.73      9744

[[1519  755]
 [2026 5444]]


In [149]:
score = metrics.accuracy_score(yrt_test, predicted)
print("accuracy:   %0.3f" % score)

accuracy:   0.715
